In [1]:
import pandas as pd

In [30]:
users_df = pd.read_csv('../data/raw/user.csv', index_col='user_id')

In [31]:
users_df.head()

,team_id,game_id
user_id,,
10189,1664,235
10018,1690,237
10580,1394,194
10600,1908,262
10112,1592,219


In [32]:
# All the users are unique
len(users_df) == len(users_df.index.unique())

True

In [26]:
users_df[users_df['game_id']==235]['team_id'].unique()

array([1664, 1670, 1672, 1668, 1666, 1674], dtype=int64)

In [35]:
# every team participates in one game
for team_id in users_df['team_id'].unique():
    if (len(users_df[users_df['team_id']==team_id]['game_id'].unique())>1):
        raise Exception(f'Teams {team_id} participates in several games')

In [39]:
results_df = pd.read_csv('../data/raw/results.csv', index_col='id')
submit_df = pd.read_csv('../data/to_sibmit.csv', index_col='id')

In [48]:
len(results_df[results_df['Analytical thinking'].isna()|results_df['Adaptability'].isna()|results_df['Systemic thinking'].isna()|results_df['Focus'].isna()])

0

In [42]:
users_df['is_train'] = users_df.index.isin(results_df.index)
users_df['is_test'] = users_df.index.isin(submit_df.index)

In [44]:
# no interceptions
len(users_df[(users_df['is_train']==True)&(users_df['is_test']==True)])

0

In [45]:
# all the users are either test or train
len(users_df[(users_df['is_train']==False)&(users_df['is_test']==False)])

0

In [54]:
users_df.drop(columns=['is_test'], inplace=True)

In [ ]:
users_df = users_df.join(results_df, how='left')

In [92]:
users_df[users_df['team_id']==1660]

,team_id,game_id,is_train,Analytical thinking,Systemic thinking,Adaptability,Focus
user_id,,,,,,,
10746,1660,234,False,NaN,NaN,NaN,NaN
10578,1660,234,True,5.0,4.0,4.0,4.0
10011,1660,234,True,4.0,5.0,4.0,4.0
10281,1660,234,False,NaN,NaN,NaN,NaN
10582,1660,234,True,5.0,4.0,5.0,5.0


In [84]:
users_to_submit_df = users_df[users_df['is_train']==False].copy()

In [93]:
users_to_submit_df[users_to_submit_df['team_id']==1660]

,team_id,game_id,is_train,Analytical thinking,Systemic thinking,Adaptability,Focus
user_id,,,,,,,
10746,1660,234,False,5.0,4.0,4.0,4.0
10281,1660,234,False,5.0,4.0,4.0,4.0


In [97]:
users_to_submit_df[['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']] =\
    users_to_submit_df.apply(lambda user: users_df[(users_df['team_id']==user['team_id'])&(users_df['is_train']==True)][['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']].mean().round(), axis = 1)
users_to_submit_df = users_to_submit_df.fillna(4.0)

In [101]:
submit_df.drop(columns=['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus'])\
    .join(users_to_submit_df[['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']]).to_csv('../data/submission/By other teammates average.csv')

In [150]:
team_points_df = pd.read_csv('../data/raw/team_point.csv')

In [151]:
team_points_df.head()

,team_id,category_id,period,score,place
0,1948,2,0,16,2
1,1934,3,4,45,6
2,1894,4,8,24,4
3,1688,4,0,27,4
4,1592,4,2,21,5


In [152]:
team_points_df[(team_points_df['category_id']==1)&(team_points_df['period']==4)]

,team_id,category_id,period,score,place
27,1374,1,4,1,3
34,1964,1,4,21,1
41,1332,1,4,37,2
67,1772,1,4,-14,6
71,1510,1,4,-15,6
...,...,...,...,...,...
10798,1464,1,4,42,1
10889,1312,1,4,-9,4
10901,1968,1,4,-8,2
10972,1316,1,4,73,2


In [156]:
team_points_df = team_points_df.merge(users_df, on='team_id')

In [157]:
len(team_points_df)

29190

In [159]:
team_points_df.min()

team_id                 1248
category_id                1
period                     0
score                    -69
place                      1
game_id                  182
is_train               False
Analytical thinking      1.0
Systemic thinking        1.0
Adaptability             1.0
Focus                    1.0
dtype: object

In [160]:
team_points_df.max()

team_id                1926
category_id               6
period                    4
score                   436
place                     6
game_id                 264
is_train               True
Analytical thinking     6.0
Systemic thinking       6.0
Adaptability            6.0
Focus                   6.0
dtype: object